In [1]:
import numpy as np
import random
import json
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import tokenize
from model import NeuralNet
import nltk
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.wrappers import FastText
from selenium import webdriver
from pprint import pprint
import requests 
from bs4 import BeautifulSoup
import unicodedata
import pandas as pd
import ast

import nltk
from nltk.corpus import stopwords
import random

In [2]:
with open("Question & Answer.json",encoding='utf-8', errors='ignore') as json_data:
     chats = json.load(json_data, strict=False)

In [ ]:
len(chats)

In [3]:
EMBEDDING_FILE='glove.6B.100d.txt'
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
model__wv = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf8"))

In [4]:
vocabulary = model__wv.keys()
len(vocabulary)

400000

In [4]:
stop_words = set(stopwords.words('english')) 
def processQues(chat):
    chat = chat.lower()
    chat = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',chat)
    chat = re.sub('@[^\s]+','',chat)
    chat = re.sub('[\s]+', ' ', chat)
    chat = re.sub(r'#([^\s]+)', r'\1', chat)
    chat = re.sub(r'[\.!:\?\-\'\"\\/]', r'', chat)
    chat = chat.strip('\'"')
    return chat

In [8]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for con in chats:
    tag = con['tags']
    # add to tag list
    tags.append(tag[0])
    ques=con["question"]
    ques=processQues(ques)
    # tokenize each word in the sentence
    w = tokenize(ques)
    w = [wd for wd in w if wd not in stop_words]
    all_words.extend(w)
    # add to xy pair
    xy.append((w, tag[0]))

In [9]:
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [ ]:
tags

In [ ]:
xy[0]

In [10]:
pairs=[]
for i in xy:
    vec=i[0]
    cls=i[1]
    document = [word for word in vec if word in model__wv.keys()]
    if (len(document)==0):
        pass
    else:
        pairs.append((document,cls))

In [11]:
len(pairs)

136950

In [12]:
X_train = []
y_train = []
for (pattern_sentence, tag) in pairs:
    pat_list=[]
    for i in pattern_sentence:
        pat_list.append(model__wv[i])
    vect=np.mean(pat_list, axis=0)
    X_train.append(vect)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [11]:
X_train[0], y_train[0]

(array([-0.17040172,  0.55024934,  0.17760606, -0.07317873, -0.03597808,
        -0.4012602 , -0.05297029,  0.1048191 ,  0.08812226, -0.07955843,
         0.00394704, -0.26159886,  0.14613499,  0.38166744,  0.07533858,
        -0.22742088,  0.2346472 , -0.00942885, -0.04003692, -0.02086451,
        -0.21291885, -0.21882844, -0.27487144,  0.29213735,  0.13796379,
         0.07480128,  0.10810026, -0.56742305, -0.15788008, -0.15723951,
         0.18862203,  0.45313588, -0.28787237, -0.0961435 ,  0.21426557,
         0.15385877, -0.04557906,  0.21416788,  0.01523628, -0.10103815,
        -0.00154607, -0.12695557, -0.21169503, -0.52027225, -0.28858867,
         0.00707572,  0.06867473,  0.04647522, -0.12436955, -0.5247038 ,
        -0.1560386 ,  0.2905142 , -0.16276036,  0.40633008, -0.05235162,
        -0.6100577 ,  0.20341988, -0.41187456,  0.7008843 ,  0.0484    ,
        -0.044756  ,  0.46844307, -0.05985422,  0.40567446,  0.42498663,
        -0.16379993,  0.3721475 ,  0.02224429,  0.1

In [13]:
#np.save( 'trainX.npy' , X_train )
#np.save( 'trainY.npy' , y_train )
len(X_train[610]), len(tags)

(100, 13)

In [14]:
# Hyper-parameters 
num_epochs = 50
batch_size = 5000
learning_rate = 0.0001
input_size = len(X_train[0])
hidden_size = 64
output_size = len(tags)
print(input_size, output_size)

100 13


In [15]:
tags

['bariatrics',
 'breast surgery',
 'cardiac electrophysiology',
 'cardiology',
 'child psychiatry',
 'clinical genetics',
 'clinical lipidology',
 'clinical psychology',
 'colon and rectal surgery',
 'critical care',
 'dentistry',
 'dermatology',
 'wound care']

In [16]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [17]:
dataset = ChatDataset()

In [18]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,output_size):
        super(NeuralNet, self).__init__()
        self.fc1=nn.Linear(input_size,128)
        self.fc2=nn.Linear(128,64)
        self.fc3=nn.Linear(64,32)
        self.fc4=nn.Linear(32,output_size)
        self.relu=nn.ReLU()
        self.drop=nn.Dropout(p=0.4)
        self.BatchNorm2=nn.BatchNorm1d(64)
        self.BatchNorm3=nn.BatchNorm1d(32)
        
    def forward(self,X):
        out=self.fc1(X)
        out=self.relu(out)
        out=self.drop(out)
        
        out=self.fc2(out)
        out=self.relu(out)
        out=self.BatchNorm2(out)
        out=self.drop(out)
        
        out=self.fc3(out)
        out=self.relu(out)
        out=self.BatchNorm3(out)
        out=self.drop(out)
        
        out=self.fc4(out)
        
        return out

In [21]:
model_4FC = NeuralNet(input_size, output_size)

In [22]:
output_size

13

In [23]:
input_size

100

In [24]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # here softmax is inbuilt so no need to apply it in neural network
optimizer = torch.optim.Adam(model_4FC.parameters(), lr=learning_rate)

In [25]:
# Train the model
count=0
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        #words = words
        labels = labels.long()
        
        # Forward pass
        outputs = model_4FC(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count+=1
        if (count+1) % 50 ==0:
            print(count)
        
    if (epoch+1) % 1 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 2.5966
49
Epoch [2/50], Loss: 2.4619
Epoch [3/50], Loss: 2.3262
99
Epoch [4/50], Loss: 2.1219
Epoch [5/50], Loss: 2.0092
149
Epoch [6/50], Loss: 1.8724
Epoch [7/50], Loss: 1.7571
199
Epoch [8/50], Loss: 1.6356
249
Epoch [9/50], Loss: 1.5884
Epoch [10/50], Loss: 1.5086
299
Epoch [11/50], Loss: 1.4052
Epoch [12/50], Loss: 1.4200
349
Epoch [13/50], Loss: 1.3778
Epoch [14/50], Loss: 1.2524
399
Epoch [15/50], Loss: 1.2417
Epoch [16/50], Loss: 1.1867
449
Epoch [17/50], Loss: 1.1821
499
Epoch [18/50], Loss: 1.1321
Epoch [19/50], Loss: 1.1330
549
Epoch [20/50], Loss: 1.0911
Epoch [21/50], Loss: 1.0334
599
Epoch [22/50], Loss: 1.0011
Epoch [23/50], Loss: 0.9910
649
Epoch [24/50], Loss: 1.0083
699
Epoch [25/50], Loss: 0.9620
Epoch [26/50], Loss: 0.9598
749
Epoch [27/50], Loss: 0.9195
Epoch [28/50], Loss: 0.8937
799
Epoch [29/50], Loss: 0.8811
Epoch [30/50], Loss: 0.8544
849
Epoch [31/50], Loss: 0.8672
Epoch [32/50], Loss: 0.8397
899
Epoch [33/50], Loss: 0.8478
949
Epoch [34/5

In [27]:
model_4FC.parameters

<bound method Module.parameters of NeuralNet(
  (fc1): Linear(in_features=100, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=13, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.4, inplace=False)
  (BatchNorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BatchNorm3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)>

In [31]:
data = {
"model_state": model_3FC.state_dict(),
"input_size": input_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "doc_data.pth"
torch.save(data, FILE)

In [32]:
print(f'training complete. file saved to {FILE}')

training complete. file saved to doc_data.pth


## Testing time

In [38]:
chats[0]

{'question': 'zirconium dental implants. how common is it used now. is there any advantages or benefits over titanium implants. cons & pros please. thanks.',
 'short_answer': '\nDental implants\n',
 'answer': 'a majority of the dental implants placed are titanium. they are highly successful with many years use ; many studies much lower in cost ; have many restorative options. zirconia implants are newer fewer studies on success lesser restorative options. however they can be more aesthetic in certain anterior(front) situations. let your dentist/oral surgeon chose what they feel will be best for you.',
 'tags': ['dentistry'],
 'url': 'https://www.healthtap.com/user_questions/1160508-zirconium-dental-implants-how-common-is-it-used-now-is-there-any-advantages-or-benefits-over-titan'}

In [5]:
#FILE = "doc_data.pth"
#data = torch.load(FILE)

In [28]:
'''input_size = data["input_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]'''

'input_size = data["input_size"]\noutput_size = data["output_size"]\nall_words = data[\'all_words\']\ntags = data[\'tags\']\nmodel_state = data["model_state"]'

In [29]:
#model = NeuralNet(input_size, output_size)
#model.load_state_dict(model_state)
model_4FC.eval()

NeuralNet(
  (fc1): Linear(in_features=100, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=13, bias=True)
  (relu): ReLU()
  (drop): Dropout(p=0.4, inplace=False)
  (BatchNorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BatchNorm3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [30]:
intent_dict={}
for intent in tags:
    int_list=[]
    for chat in chats:
        if(chat["tags"][0]==intent):
            int_list.append(chat["short_answer"])
    intent_dict[intent]=int_list

In [ ]:
bot_name = "Robo: "
print("----------------Your Personal Doctor! (type 'bye' to exit)-------------------")
while True:
    sentence = input("You: ")
    if sentence == "bye":
        break
    sentence=processQues(sentence)
    sentence = tokenize(sentence)
    sentence = [wd for wd in sentence if wd not in stop_words]
    #print(sentence)
    pat_list=[]
    for i in sentence:
        pat_list.append(model__wv[i])
    vect=np.mean(pat_list, axis=0)
    X = vect.reshape(1, vect.shape[0])
    X = torch.from_numpy(X)

    output = model_4FC(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]
    #print(tag)
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print(prob.item())
    if prob.item() > 0.40:
        print(f"{bot_name}: {random.choice(intent_dict[tag])}")
    else:
        print(f"{bot_name}: I do not understand...")